In [41]:
import pandas as pd # data frames
import numpy as np

In [42]:
# read data
df = pd.read_csv("https://raw.githubusercontent.com/askoshiyama/mli-cohort3/master/boston.csv")
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,T1
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


Repeated K Fold CV (5 outer, 5 inner, repeat 5 times):
RepeatedKFold repeats K-Fold n times. It can be used when one requires to run KFold n times, producing different splits in each repetition.
https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators

In [43]:
# defining vars
input_vars = ["V1", "V2", "V3", "V4", "V5", "V6", "V7",
              "V8", "V9", "V10", "V11", "V12", "V13"]
target = ["T1"]

In [44]:
X = df.drop(labels=target, axis=1)
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33


In [45]:
y = df.drop(labels=input_vars, axis=1)
y.head()

,T1
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [81]:
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from interpret.glassbox import ExplainableBoostingRegressor 
from xgboost import XGBRegressor
from sklearn.svm import SVR

# define the models for selection along with their hyperparameters
# we try to find meaningful ranges for each hyperparameter. Among the models chosen for selection,
# we also include an (MLP) neural network as it presents the most recent advancement in the supervised learning domain.
models_and_parameters = {
    'dummy': (DummyRegressor(), {}),
    'linReg': (LinearRegression(), {}),
    'ridge': (Ridge(), 
              {'alpha': np.linspace(0.00001, 0.8, num=50)}),
    'svr': (SVR(kernel='rbf'),
              {'C': [0.01, 0.05, 0.1, 0.5, 1], 
               'gamma': [0.001, 0.0001, 0.00001]}),
    'krr': (KernelRidge(), 
              {'kernel': ["poly","rbf"],
               'degree': [2,3,4],
               'alpha': np.linspace(0.00001, 0.8, num=50)}),
    'dtr': (DecisionTreeRegressor(),
              {"min_samples_split": [2, 10, 20, 40], 
               "max_depth": [2, 6, 8],
               "min_samples_leaf": [1, 20, 40, 100],
               "max_leaf_nodes": [None, 5, 20, 100]}),
    'rf': (RandomForestRegressor(n_estimators=100),
              {'max_depth': [10, 50, 100, 150, 200]}),
    'gbr': (GradientBoostingRegressor(),
              {"max_depth": [2, 6, 8],
               "learning_rate": [0.1, 0.01, 0.001],
               "n_estimators": [50, 100, 150]}),
    'ebr': (ExplainableBoostingRegressor(), {}),
    'xgbr': (XGBRegressor(), 
              {"max_depth": [2, 6, 8],
               "learning_rate": [0.1, 0.01, 0.001],
               "n_estimators": [50, 100, 150]}),
    'mlp': (MLPRegressor(early_stopping=True),
                {'hidden_layer_sizes': [(64,),(128,)],
                 'activation': ['relu'],
                 'solver':['lbfgs', 'adam'], 
                 'alpha':[0.0001],
                 'batch_size':['auto'], 
                 'learning_rate':['adaptive', 'constant'],
                 'learning_rate_init':[0.1, 0.01, 0.001], 
                 'max_iter':[1600]})
}

In [82]:
from sklearn import metrics

# performance metrics
perf_metrics = {"MAE": metrics.mean_absolute_error, 
                "EVS": metrics.explained_variance_score, 
                "MSE": metrics.mean_squared_error,
                "MDAE": metrics.median_absolute_error, 
                "R2": metrics.r2_score
               }
metric_greater_is_better = \
               {"MAE": False, 
                "EVS": True, 
                "MSE": False,
                "MDAE": False, 
                "R2": True
               }
scorer_metrics = {}
for pf in perf_metrics:
    scorer_metrics[pf] = metrics.make_scorer(score_func=perf_metrics[pf], greater_is_better=metric_greater_is_better[pf])

In [83]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold, KFold, cross_val_score, GridSearchCV
import numpy as np
from sklearn import metrics
from time import time


#######################################
# configuration
REPEAT = 5
SEED = None  # will ensure different results on each iteration
K_FOLD_NUM = 5
REFIT_METRIC = 'MSE'
#######################################

# use nested cross-validation for grid search and validation
# creates 5 folds for estimating generalization error, repeat 
outer_folds = RepeatedKFold(n_repeats=REPEAT, n_splits=K_FOLD_NUM, random_state=SEED)

# when we train on a certain fold, we use a second cross-validation
# split in order to choose hyperparameters
inner_folds = KFold(n_splits=K_FOLD_NUM, random_state=SEED, shuffle=True)

i, z, innercv_results, inner_best_model = 0, 0, {}, {}
df_outer_results = pd.DataFrame(index=[0], columns=["Model", "Time"] + list(perf_metrics.keys()))

X_arr, y_arr = X.values, y.values

# outer loop
for (out_train, out_test) in outer_folds.split(X_arr, y_arr):
    print(f"Repetition {int(i / K_FOLD_NUM) + 1} Fold {i % K_FOLD_NUM + 1}")
    i += 1  # iteration
    # separation: train, test
    Xtrain, Xtest = X_arr[out_train], X_arr[out_test]
    ytrain, y_test = y_arr[out_train], y_arr[out_test]
    
    # inner loop -- all models
    fold_id = "fold_" + str(i)
    innercv_results[fold_id] = {}
    inner_best_model[fold_id] = {}
    start, end = [], []
    
    for model_name, (model, params) in models_and_parameters.items():
        start.append(time())
        print("\t", model_name)
        innercv_results[fold_id][model_name] = GridSearchCV(estimator=model, 
                                                            param_grid=params, 
                                                            scoring=scorer_metrics, 
                                                            iid=False,
                                                            cv=inner_folds, 
                                                            refit=REFIT_METRIC)    
        innercv_results[fold_id][model_name].fit(Xtrain, ytrain.ravel())
        inner_best_model[fold_id][model_name] = innercv_results[fold_id][model_name].best_estimator_
        end.append(time())        
    
    # outer loop prediction    
    for v, (model_name, (model, _)) in enumerate(models_and_parameters.items()):
        # prediction
        y_test_pred = inner_best_model[fold_id][model_name].predict(Xtest)
        df_outer_results.loc[z, "Model"] = model_name
        df_outer_results.loc[z, "Time"] = end[v] - start[v]
        
        # compute performance metrics
        for pf in perf_metrics.keys():
            df_outer_results.loc[z, pf] = perf_metrics[pf](y_test, y_test_pred)
        z += 1        

# final organisation
df_outer_results[list(perf_metrics.keys()) + ["Time"]] = df_outer_results[list(perf_metrics.keys()) + ["Time"]].astype(float)
print("done.")

Repetition 1 Fold 1
	 dummy
	 linReg
	 ridge
	 svr
	 krr


C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anacon

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anacon

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=2.68105e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_mod

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anacon

...

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=6.42927e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=5.973e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=8.04831e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=6.07526e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=7.41021e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\env

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=8.01235e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=8.90388e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=9.1038e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=8.72143e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=9.94435e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\en

	 dtr
	 rf
	 gbr
	 ebr
	 xgbr
[13:56:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:

[13:56:29] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:29] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:29] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:29] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:29] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:29] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:29] WARNI

[13:56:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:32] WARNI

In [84]:
df_outer_results.pivot_table(index=["Model"], aggfunc="mean")

,EVS,MAE,MDAE,MSE,R2,Time
Model,,,,,,
dtr,7.524837e-01,2.917355,2.081760,20.752252,0.749354,3.253037
dummy,-1.776357e-17,6.673926,4.827640,84.931233,-0.018457,0.014931
ebr,8.387097e-01,2.478392,1.749318,13.509972,0.837084,38.220235
gbr,8.670389e-01,2.228622,1.593591,11.083170,0.865863,14.491997
krr,8.122277e-01,2.586518,1.830955,15.561584,0.809682,38.343112
linReg,7.146486e-01,3.410593,2.538652,23.909132,0.711330,0.017043
mlp,7.920404e-01,2.876883,2.100930,17.340479,0.789615,307.999464
rf,8.682230e-01,2.227240,1.529379,11.050778,0.866357,8.962548
ridge,7.147135e-01,3.405303,2.519364,23.902860,0.711378,0.726414


In [85]:
# pick the best model according to the refit metric
best_models_mean = df_outer_results.pivot_table(index=["Model"], aggfunc="mean")
best_model_refit_metric = best_models_mean[REFIT_METRIC].idxmin()
best_model_entry = best_models_mean[best_models_mean.index == best_model_refit_metric]
best_model_entry

,EVS,MAE,MDAE,MSE,R2,Time
Model,,,,,,
xgbr,0.885014,2.127671,1.542749,9.614056,0.883473,10.212758


In [86]:
best_model_name = best_model_entry.index.values[0]
best_model_name

'xgbr'

Using the MSE refit metric, XGBRegressor is the best model. Now we pick the model that achieved the lowest MSE score:

In [87]:
best_model_results = df_outer_results[df_outer_results['Model'] == best_model_refit_metric]
best_model_results

,Model,Time,MAE,EVS,MSE,MDAE,R2
9,xgbr,21.187455,1.852320,0.910994,6.842352,1.552854,0.909056
20,xgbr,20.209848,2.719013,0.803165,17.064287,2.021964,0.803162
31,xgbr,20.912170,2.214827,0.904227,8.576428,1.666195,0.903542
42,xgbr,7.889901,2.275593,0.842174,13.185410,1.675485,0.841993
53,xgbr,8.367820,1.904904,0.912892,7.713824,1.334703,0.909839
64,xgbr,7.536877,2.335062,0.891098,10.312566,1.560225,0.886573
75,xgbr,7.925833,1.909733,0.919169,7.482796,1.218889,0.919031
86,xgbr,7.544226,2.048150,0.862442,13.165570,1.371547,0.861824
97,xgbr,7.660515,2.167992,0.862012,10.227807,1.427867,0.861040
108,xgbr,7.535844,2.001632,0.875688,8.676734,1.319316,0.873385


In [88]:
idxmin = best_model_results[REFIT_METRIC].idxmin()

fold_id = int(idxmin / len(models_and_parameters))
best_estimator = innercv_results['fold_' + str(fold_id)][best_model_name].best_estimator_
best_params = innercv_results['fold_' + str(fold_id)][best_model_name].best_params_

In [90]:
best_estimator

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [93]:
best_params

{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100}

Findings/Discussion/Arguments: 
To perform the model selection, we add a variety of models, including linear and non-linear models. In addition, we complement the competition with a neural network, i.e. a basic MLP with just one hidden layer, which is also a non-linear model. 
We then use nested cross-validation in order to perform the optimization (i.e. grid-search) and the test inference on different parts of the underlying dataset. In the presence of multiple metrics, it is difficult to select "the best" model. We pick the MSE metric to guide the decision, as it is well suited for a regression problem. 
We observe that the MLP has a significant longer training time, due to the larger hyperparameter search space and the nature of the computation (esp. backpropagation), but likely also due to the parameter 'max_iter=1600'. We note that the dataset with 500 samples is rather small to train a neural network. In general, the more data the better. So the performance of the MLP could likely improve as the dataset increases. However, if the dataset is simply increased by copying with no new samples added, we conjecture that this would only serve the chances of the MLP overfitting on the training data. 
In terms of MSE performance, a few models stand out: below an MSE of 12 there is only gradient boosting regressor, random forest regressor and XGBoost regressor, which is also a gradient boosting method. We expected the MLP to perform on par with the gradient boosting models, but it turned out to perform worse. We conjecture that the performance of the MLP could be increased with at least another hidden layer and, as mentioned before, by increasing the dataset with more samples. 
In this exact scenario, however, our conclusion is that the XGBoost regressor is the best model, with a maximum depth of 6 and 100 estimators. This also exemplifies that not always do greater maximum depths and more estimators necessarily yield a better model performance, one possible reason being increased variance with increasing tree depth.
In summary, the model selection should be conducted with hyperparameter search spaces as large as possible to squeeze out the best model configuration indeed. Here, our search space was rather small. All in all, we belive with the XGBoost regressor we have found a fairly good model for the boston house price prediction task to build upon, which is also reflected in a good 88.3% R2 score.